In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

def create_unet_model(input_shape, num_classes):
    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Downsampling path
    x = inputs
    skip_connections = []
    for i in range(5):
        filters = 2**(8+i)
        x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, padding="same", activation="relu")(x)
        x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, padding="same", activation="relu")(x)
        skip_connections.append(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)

    # Upsampling path
    for i in range(5):
        filters = 2**(7-i)
        x = tf.keras.layers.Conv2DTranspose(filters=filters, kernel_size=2, strides=2, padding="same")(x)
        x = tf.keras.layers.concatenate([x, skip_connections.pop()])
        x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, padding="same", activation="relu")(x)
        x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, padding="same", activation="relu")(x)

    # Output Layer
    outputs = tf.keras.layers.Conv2D(num_classes, 1, activation="softmax")(x)

    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model


2022-12-26 09:20:57.515015: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
input_shape = (512,512,1)
num_classes = 2
model = create_unet_model(input_shape, num_classes)
model.summary()

2022-12-26 09:21:00.471297: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-26 09:21:00.472350: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-12-26 09:21:00.484825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-26 09:21:00.484936: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.665GHz coreCount: 80 deviceMemorySize: 11.76GiB deviceMemoryBandwidth: 849.46GiB/s
2022-12-26 09:21:00.484951: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-12-26 09:21:00.495890: I tensorflow/stream_executor/platfor

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 256 2560        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 256 590080      conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 256 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [3]:
import rasterio

# Load the data
image_path = "dem_data.tif"
labels_path = "label_data_int.tif"

with rasterio.open(image_path, "r") as dataset:
    image = dataset.read(1, window=rasterio.windows.Window(0, 0, 512, 512))

with rasterio.open(labels_path, "r") as dataset:
    labels = dataset.read(1, window=rasterio.windows.Window(0, 0, 512, 512))
# Preprocess the data
image = tf.cast(image, tf.float32) / tf.cast(tf.reduce_max(image), tf.float32)
labels = tf.cast(labels, tf.int8)
image = tf.reshape(image, shape=[1, image.shape[0], image.shape[1], 1])
labels = tf.reshape(labels, shape=[1, labels.shape[0], labels.shape[1], 1])
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(image, labels, epochs=5, batch_size=32)

# Save the model
model.save("model.h5")

2022-12-26 09:21:05.755189: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-12-26 09:21:05.755464: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3701010000 Hz


Epoch 1/5


2022-12-26 09:21:06.538138: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-12-26 09:31:33.938563: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-12-26 09:31:33.989386: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-12-26 09:31:34.460418: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


1/1 [==============================] - 721s 721s/step - loss: nan - accuracy: 0.9575
Epoch 2/5
1/1 [==============================] - 0s 385ms/step - loss: nan - accuracy: 0.9575
Epoch 3/5
1/1 [==============================] - 0s 390ms/step - loss: nan - accuracy: 0.9575
Epoch 4/5
1/1 [==============================] - 0s 385ms/step - loss: nan - accuracy: 0.9575
Epoch 5/5
1/1 [==============================] - 0s 389ms/step - loss: nan - accuracy: 0.9575


2022-12-26 09:33:08.971437: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 301989888 exceeds 10% of free system memory.
2022-12-26 09:33:09.271362: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 603979776 exceeds 10% of free system memory.
2022-12-26 09:33:09.990158: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 301989888 exceeds 10% of free system memory.
2022-12-26 09:33:10.086545: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 603979776 exceeds 10% of free system memory.
2022-12-26 09:33:10.474427: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 301989888 exceeds 10% of free system memory.


In [4]:
# Use the model to make predictions
model = tf.keras.models.load_model("model.h5")

# Use the trained model to make predictions
predictions = model.predict(image)
print(predictions[0,:,:,1])

# Save the predictions using rasterio
with rasterio.open("predictions.tif", "w", driver="GTiff", width=512, height=512, count=1, dtype=rasterio.float32) as dst:
    dst.write(predictions[0,:,:,1].astype(rasterio.float32), 1)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


/home/bb/.conda/envs/ML/lib/python3.9/site-packages/rasterio/__init__.py:230: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = writer(path, mode, driver=driver,
